## 20250823数据正先导部分分析

### 加载2d定位结果数据

In [1]:
import pandas as pd
import numpy as np

result = pd.read_csv('results/20250823172542_1505_result_yld_5.95e8_8.95e8_hann_2048_256_bandpass_hann_30e6_80e6.txt', sep=r'\s+')

# 对2d结果进行过滤筛选
filtered_result = result[
    (abs(result['t123']) < 1) &  
    (abs(result['Rcorr']) > 0.35) &  
    (result['Start_loc'] < 8.96e8) & 
    (result['Start_loc'] > 7.96e8)
    ]
filtered_result


,Start_loc,peak,t12,t13,t23,cos_alpha_opt,cos_beta_opt,Azimuth,Elevation,Rcorr,t123
29068,796981800,2048,-64.1,-58.2,5.9,-0.551217,0.187921,288.825289,54.382601,0.370087,-0.0
29156,798302907,2048,-94.0,-38.0,56.0,-0.959091,-0.194493,258.536508,11.871233,0.363793,0.0
29157,798303198,2048,-94.4,-38.1,56.4,-0.963857,-0.196467,258.479011,10.366475,0.436504,0.1
29158,798303496,2048,-94.4,-38.0,56.8,-0.965643,-0.199061,258.352040,9.616749,0.375578,0.4
29236,798844575,2048,-45.1,-70.1,-24.2,-0.298927,0.417363,324.388735,59.111386,0.384089,0.8
...,...,...,...,...,...,...,...,...,...,...,...
47228,894489720,2048,-54.1,26.9,81.5,-0.709715,-0.598798,229.845158,21.786042,0.598681,0.5
47229,894506622,2048,-19.7,69.3,88.8,-0.446027,-0.806771,208.936209,22.800948,0.364806,-0.2
47232,894622761,2048,-20.1,70.0,90.0,-0.453341,-0.816687,209.034539,20.920746,0.799153,-0.1
47233,894850924,2048,-21.1,69.2,90.2,-0.462284,-0.814828,209.567967,20.474152,0.454664,-0.1


### 分析梯级间隔

In [2]:
# 选出正先导数据
start_loc = 7.96e8
end_loc = 8.96e8
positive_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
    ]
time_diffs = np.diff(positive_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = positive_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:", corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 3000
filtered_positive_time_diffs = time_diffs[time_diffs > thea]
filtered_positive_time_diffs = filtered_positive_time_diffs * 5 / 1e3
print(f"时间间隔大于{thea}的个数有：{len(filtered_positive_time_diffs)}")
print(f"正先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",
      pd.DataFrame(data=filtered_positive_time_diffs).describe())

最大的间隔位于: 860652610
时间间隔大于3000的个数有：690
正先导 796000000.0-896000000.0 阶段的时间间隔分布情况：
                   0
count    690.000000
mean     702.062674
std     1355.173704
min       15.080000
25%       32.530000
50%      113.385000
75%      755.163750
max    13837.425000


In [3]:
# 计算梯级步长
## Step1: 拿到每个间隔对应的位置范围
filtered_indices = np.where(time_diffs > thea)
filtered_start_loc = positive_lead_result.Start_loc.iloc[filtered_indices[0]].values    # .values 将结果转为Numpy数组
start_locs = filtered_start_loc[:-1]  # 去掉最后一个元素
end_locs = filtered_start_loc[1:]    # 去掉第一个元素
# 拼接为元组列表
loc_tuples = list(zip(start_locs, end_locs))

durations = []
# Step2: 三维结果中筛选出对应的结果
for i, loc_tuple in enumerate(loc_tuples):
    durations.append((loc_tuple[1]-loc_tuple[0])*5/1e3)

durations_df = pd.DataFrame(data=durations)
print(f"正先导 {start_loc}-{end_loc} 阶段的梯级持续时间分布情况：\n",durations_df.describe())
        

正先导 796000000.0-896000000.0 阶段的梯级持续时间分布情况：
                   0
count    689.000000
mean     710.225864
std     1353.754349
min       15.245000
25%       42.675000
50%      134.315000
75%      758.180000
max    13841.185000
